# Pre-Processing

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.stem import snowball

In [ ]:
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stop_words_list = stopwords.words('english')
stemmer = snowball.EnglishStemmer()

In [ ]:
def clean (text, tokenizer, stop_words_list,stemmer):
    lower_text = text.lower()
    token_text = tokenizer.tokenize(lower_text)
    stop_text = [word for word in token_text if word not in stop_words_list]
    stem_text = [stemmer.stem(word) for word in stop_text]
    return stem_text
    

In [ ]:
text_data = report.apply(lambda x: clean(x,tokenizer, stop_words_list,stemmer))

In [ ]:
report['processed_report'] = text_data

In [ ]:
cv = CountVectorizer(stop_words='english',min_df=.05,max_df=.9,)
cv.fit_transform(text_data)

In [ ]:
cv.get_feature_names_out()